In [2]:
import yfinance as yf
import pandas as pd
import numpy as np
import ta
import matplotlib.pyplot as plt

In [30]:
class Backtester:
    def __init__(self, symbol):
        self.symbol = symbol
        self.df = yf.download(self.symbol, start = '2019-01-01')
        if self.df.empty:
            print("No data pulled")
        else:
            self.calc_indicators()
            self.generate_signal()
            self.filter_sig_overlap()
    
    def calc_indicators(self):

        self.df['ma_20'] = self.df.Close.rolling(20).mean()
        self.df['vol'] = self.df.Close.rolling(20).std()
        self.df["upper_bb"] = self.df.ma_20+(2*self.df.vol)
        self.df["lower_bb"] = self.df.ma_20-(2*self.df.vol)
        self.df["rsi"] = ta.momentum.rsi(self.df.Close, window = 6)
        self.df.dropna(inplace=True)
        
    def generate_signal(self):
        conditions = [(self.df.rsi <30) & (self.df.Close<self.df.lower_bb),
             (self.df.rsi>70) & (self.df.Close>self.df.upper_bb)]
        choices = ['Buy', "Sell"]
        self.df['signal']= np.select(conditions,choices)
        self.df.signal = self.df.signal.shift()
        self.df.dropna(inplace = True)
    
    def filter_sig_overlap(self):
        # simple without stoploss 
#         self.df['shifted_Close'] = self.df.Close.shift()
        position = False
        buydates, selldates = [],[]
        
        for index, row in self.df.iterrows():
            if not position and row['signal'] == 'Buy':
                buydates.append(index)
                position = True

            if position and row['signal'] == "Sell":
                selldates.append(index)
                position = False
        
        self.buy_arr = self.df.loc[buydates].Open
        self.sell_arr = self.df.loc[selldates].Open

In [31]:
bt_instance = Backtester('SQ')

[*********************100%***********************]  1 of 1 completed


In [32]:
bt_instance

In [33]:
bt_instance.symbol

'SQ'

In [34]:
bt_instance.df

,Open,High,Low,Close,Adj Close,Volume,ma_20,vol,upper_bb,lower_bb,rsi,signal
Date,,,,,,,,,,,,
2019-01-31,70.029999,73.860001,69.980003,71.349998,71.349998,15679400,67.367000,5.989839,79.346678,55.387322,52.662931,0
2019-02-01,71.050003,71.279999,69.720001,70.800003,70.800003,12677800,68.286000,4.883736,78.053472,58.518528,50.687025,0
2019-02-04,71.000000,72.620003,70.400002,71.750000,71.750000,10582100,68.964500,4.315380,77.595259,60.333741,54.245314,0
2019-02-05,73.000000,75.169998,72.269997,72.279999,72.279999,19150500,69.542500,3.907931,77.358362,61.726637,56.353753,0
2019-02-06,72.309998,73.279999,70.820000,72.669998,72.669998,10068500,70.020000,3.657287,77.334574,62.705426,58.060310,0
...,...,...,...,...,...,...,...,...,...,...,...,...
2023-01-05,64.849998,65.559998,63.019001,64.709999,64.709999,11336200,63.609000,3.337662,70.284323,56.933676,57.555529,0
2023-01-06,65.279999,69.500000,63.360001,68.989998,68.989998,15518900,64.005000,3.487059,70.979118,57.030881,72.127639,0
2023-01-09,69.699997,70.610001,68.440002,69.059998,69.059998,12987600,64.289499,3.660335,71.610170,56.968829,72.314188,0


In [36]:
bt_instance.buy_arr

Date
2019-04-22     70.050003
2019-08-05     66.199997
2019-12-24     62.799999
2020-03-09     67.209999
2020-11-02    158.000000
2021-01-27    204.009995
2021-03-09    215.369995
2021-05-05    237.500000
2021-09-09    256.000000
2022-04-22    108.110001
2022-09-21     61.360001
Name: Open, dtype: float64

In [37]:
bt_instance.sell_arr

Date
2019-06-10     69.570000
2019-11-20     66.360001
2020-01-09     68.220001
2020-05-07     69.860001
2020-11-24    208.000000
2021-02-09    256.579987
2021-04-14    274.130005
2021-06-18    235.029999
2022-03-21    138.360001
2022-07-21     73.309998
2022-11-14     70.250000
Name: Open, dtype: float64